# NeuroData Design: Sprint 1
##### Patrick Myers, December 14

This file is the R portion of my Sprint 1 goal. Part of our team focused on generating graph statistics to be used as possible features for clustering. The feature space generated by these statistics is rather large and not all features will be useful for our analysis. Therefore, a method for choosing useful features out of this large pool was needed. Early attempts were general methods, mostly looking at the variance within the features themselves. These methods proved inefficient for clustering. The newer solution is presented here, with the help of NDD's multiscale graph correlation (MGC). Now features are chosen based on their correlation with some distinguishable label in the dataset (Age and Sex for the HBN dataset used here, but individual patient could be used for the HNU1 dataset). This code will hopefully serve as a filtering method between the feature generation team and the clustering team.

Since the code will exist within Gigantum, setting up packages is necessary. Adding the MGC package was done manually.

In [2]:
.libPaths("Packages")
install.packages(c('ggplot2', 'reshape2', 'Rmisc', 'devtools', 'testthat', 'knitr', 'rmarkdown', 'latex2exp', 'MASS'), 'Packages')
install.packages('abind', 'Packages')
library(mgc)
library(reshape2)
library(ggplot2)

also installing the dependencies ‘colorspace’, ‘fansi’, ‘utf8’, ‘ps’, ‘labeling’, ‘munsell’, ‘RColorBrewer’, ‘pillar’, ‘glue’, ‘stringi’, ‘processx’, ‘assertthat’, ‘curl’, ‘openssl’, ‘digest’, ‘gtable’, ‘lazyeval’, ‘plyr’, ‘rlang’, ‘scales’, ‘tibble’, ‘viridisLite’, ‘withr’, ‘Rcpp’, ‘stringr’, ‘callr’, ‘cli’, ‘git2r’, ‘httr’, ‘jsonlite’, ‘memoise’, ‘rstudioapi’, ‘crayon’, ‘magrittr’, ‘R6’, ‘evaluate’, ‘highr’, ‘markdown’, ‘yaml’, ‘xfun’, ‘htmltools’, ‘base64enc’, ‘mime’, ‘tinytex’

Warning message in install.packages(c("ggplot2", "reshape2", "Rmisc", "devtools", :
“installation of package ‘Rmisc’ had non-zero exit status”

Defining Y1 as sex and Y2 as age. Thresholding age at 10.5 (median of the dataset) to be a binary label showed higher correlations than leaving age in raw form.

In [6]:
set.seed(12345)
dat <- read.csv("../input/hbn_vertexstats.csv", header = TRUE)
Y1 <- dat[,2]
Y2 <- dat[,3]
Y2 <- Y2 > 10.5
Y2 <- as.numeric(Y2)

 [1] 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
[39] 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 1
[77] 0 1 0 1 1 0 0 1 1 1 1 0 1 0 0


For each vertex statistic, calculate the correlation with sex using MGC. Report the test statistic, p value, and optimal scale values.

In [4]:
zeromat <- matrix(1:288, nrow = 288, ncol=6)
mgctests <- array(0L, dim(zeromat))

count <- 1

for(i in 4:291){
  X <- dat[,i]
  Xdat <- array(as.numeric(unlist(X)), dim=c(91,1))
  YDat <- array(as.numeric(unlist(Y1)), dim=c(91,1))
  res <- mgc.test(Xdat,YDat, rep=20)
  val <- res$statMGC
  p_val <- res$pMGC
  scale <- res$optimalScale

  mgctests[count,1] = val
  mgctests[count,2] = 'Xdat'
  mgctests[count,3] = 'YDat'
  mgctests[count,4] = p_val
  mgctests[count,5] = scale$x
  mgctests[count,6] = scale$y
  count <- count +1
}


CSV files saved as input for the python code and output for demonstration

In [7]:
Xdat <- array(as.numeric(unlist(dat[,258])), dim=c(91,1))
YDat <- array(as.numeric(unlist(Y1)), dim=c(91,1))
res <- mgc.test(Xdat,YDat, rep=20)
corr_mat <- res$localCorr
write.csv(corr_mat, file = '../output/Gender_vertex_corr.csv')
write.csv(mgctests, file = '../output/mgc-test-stats_gender_vertex.csv')
write.csv(corr_mat, file = '../input/Gender_vertex_corr.csv')
write.csv(mgctests, file = '../input/mgc-test-stats_gender_vertex.csv')

Repeat analysis again, but using age as the correlating feature

In [34]:
zeromat <- matrix(1:288, nrow = 288, ncol=6)
mgctests <- array(0L, dim(zeromat))

count <- 1

#Xlist <- list(X1, X2, X3, X4, X5, X6)
for(i in 4:291){
  X <- dat[,i]
  Xdat <- array(as.numeric(unlist(X)), dim=c(91,1))
  YDat <- array(as.numeric(unlist(Y2)), dim=c(91,1))
  res <- mgc.test(Xdat,YDat, rep=20)
  val <- res$statMGC
  p_val <- res$pMGC
  scale <- res$optimalScale

  mgctests[count,1] = val
  mgctests[count,2] = 'Xdat'
  mgctests[count,3] = 'YDat'
  mgctests[count,4] = p_val
  mgctests[count,5] = scale$x
  mgctests[count,6] = scale$y
  count <- count +1
}

In [35]:
Xdat <- array(as.numeric(unlist(dat[,119])), dim=c(91,1))
YDat <- array(as.numeric(unlist(Y2)), dim=c(91,1))
res <- mgc.test(Xdat,YDat, rep=20)
corr_mat <- res$localCorr
write.csv(corr_mat, file = '../output/Age_vertex_corr.csv')

write.csv(mgctests, file = '../output/mgc-test-stats_age_vertex.csv')
write.csv(corr_mat, file = '../input/Age_vertex_corr.csv')

write.csv(mgctests, file = '../input/mgc-test-stats_age_vertex.csv')


Repeat the above procedure. Instead of calculating the correlation for each vertex, the correlation is calculated per feature (i.e. the K-Hop locality of all 48 vertices against sex).

In [36]:
set.seed(12345)
dat <- read.csv("../input/hbn_vertexstats.csv", header = TRUE)
#dat <- array(as.numeric(unlist(dat)), dim=c(91,291))
Y1 <- dat[,2]
Y2 <- dat[,3]
Y2 <- Y2 > 10.5
Y2 <- as.numeric(Y2)
X1 <- dat[,c(4:51)]
X2 <- dat[,c(52:99)]
X3 <- dat[,c(100:147)]
X4 <- dat[,c(148:195)]
X5 <- dat[,c(196:243)]
X6 <- dat[,c(244:291)]

 [1] 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
[39] 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 1
[77] 0 1 0 1 1 0 0 1 1 1 1 0 1 0 0


In [37]:
zeromat <- matrix(1:6, nrow = 6, ncol=6)
mgctests <- array(0L, dim(zeromat))

count <- 1

Xlist <- list(X1, X2, X3, X4, X5, X6)
for(i in Xlist){
  Xdat <- array(as.numeric(unlist(i)), dim=c(91,48))
  YDat <- array(as.numeric(unlist(Y1)), dim=c(91,1))
  res <- mgc.test(Xdat,YDat, rep=20)
  val <- res$statMGC
  p_val <- res$pMGC
  scale <- res$optimalScale

  mgctests[count,1] = val
  mgctests[count,2] = 'Xdat'
  mgctests[count,3] = 'YDat'
  mgctests[count,4] = p_val
  mgctests[count,5] = scale$x
  mgctests[count,6] = scale$y
  count <- count +1
}


In [39]:
Xdat <- array(as.numeric(unlist(X1)), dim=c(91,1))
YDat <- array(as.numeric(unlist(Y1)), dim=c(91,1))
res <- mgc.test(Xdat,YDat, rep=20)
corr_mat <- res$localCorr
write.csv(corr_mat, file = '../output/Gender_feature_corr.csv')

write.csv(mgctests, file = '../output/mgc-test-stats_gender_feature.csv')

write.csv(corr_mat, file = '../input/Gender_feature_corr.csv')

write.csv(mgctests, file = '../input/mgc-test-stats_gender_feature.csv')



In [40]:
zeromat <- matrix(1:6, nrow = 6, ncol=6)
mgctests <- array(0L, dim(zeromat))

count <- 1

#Xlist <- list(X1, X2, X3, X4, X5, X6)
for(i in Xlist){
  Xdat <- array(as.numeric(unlist(i)), dim=c(91,48))
  YDat <- array(as.numeric(unlist(Y2)), dim=c(91,1))
  res <- mgc.test(Xdat,YDat, rep=20)
  val <- res$statMGC
  p_val <- res$pMGC
  scale <- res$optimalScale

  mgctests[count,1] = val
  mgctests[count,2] = 'Xdat'
  mgctests[count,3] = 'YDat'
  mgctests[count,4] = p_val
  mgctests[count,5] = scale$x
  mgctests[count,6] = scale$y
  count <- count +1
}

In [41]:
Xdat <- array(as.numeric(unlist(X4)), dim=c(91,1))
YDat <- array(as.numeric(unlist(Y2)), dim=c(91,1))
res <- mgc.test(Xdat,YDat, rep=20)
corr_mat <- res$localCorr
write.csv(corr_mat, file = '../output/Age_feature_corr.csv')


write.csv(mgctests, file = '../output/mgc-test-stats_age_feature.csv')

write.csv(corr_mat, file = '../input/Age_feature_corr.csv')


write.csv(mgctests, file = '../input/mgc-test-stats_age_feature.csv')